In [1]:
from srgan import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
vgg_path = 'vgg16_notop.h5'
srgan = SRGAN(vgg_path)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
srgan.vgg_combined.get_weights()[0]

array([[[[-7.1850024e-02,  9.2915297e-03,  5.8374375e-02, ...,
           7.5503871e-02,  4.1689634e-02, -4.8872307e-03],
         [-2.1022283e-02, -5.7787400e-02, -9.4348416e-03, ...,
          -9.7819991e-02, -8.8411003e-02, -1.4400624e-02],
         [-6.8533078e-02, -4.3334812e-02,  8.6616725e-03, ...,
           3.3914432e-02,  8.3805457e-02,  9.5697120e-03]],

        [[ 3.6444619e-02, -3.2364525e-02, -7.0563652e-02, ...,
           1.2512796e-02, -3.4848556e-02,  2.4786979e-02],
         [-7.7960119e-03,  4.1621238e-02,  2.2591092e-02, ...,
           2.3578629e-03,  6.2887743e-02, -8.6168408e-02],
         [ 2.5122263e-02,  5.9960410e-03,  6.2671915e-02, ...,
           4.9903572e-02,  3.5606608e-02, -5.8281336e-02]],

        [[ 2.7932882e-02, -8.7643631e-02,  1.7155774e-02, ...,
           8.2015470e-02, -7.3620394e-02, -8.5265361e-02],
         [ 6.4643964e-02,  8.3204657e-02, -8.5551061e-02, ...,
          -1.7823681e-02, -8.2975566e-02, -6.9040731e-03],
         [ 5.7288036

In [4]:
input_dir = 'data/low/'
tgt_dir = 'data/high/'
gen = ImageDataGenerator(input_dir, tgt_dir)

In [5]:
data = gen.flow_from_directory().__next__()

In [6]:
X_low = data[0]
X_high = data[1]
srgan.train(X_low, X_high)

training discriminator (true)...



In [7]:
srgan.vgg_combined.get_weights()[0]

array([[[[-7.1850024e-02,  9.2915297e-03,  5.8374375e-02, ...,
           7.5503871e-02,  4.1689634e-02, -4.8872307e-03],
         [-2.1022283e-02, -5.7787400e-02, -9.4348416e-03, ...,
          -9.7819991e-02, -8.8411003e-02, -1.4400624e-02],
         [-6.8533078e-02, -4.3334812e-02,  8.6616725e-03, ...,
           3.3914432e-02,  8.3805457e-02,  9.5697120e-03]],

        [[ 3.6444619e-02, -3.2364525e-02, -7.0563652e-02, ...,
           1.2512796e-02, -3.4848556e-02,  2.4786979e-02],
         [-7.7960119e-03,  4.1621238e-02,  2.2591092e-02, ...,
           2.3578629e-03,  6.2887743e-02, -8.6168408e-02],
         [ 2.5122263e-02,  5.9960410e-03,  6.2671915e-02, ...,
           4.9903572e-02,  3.5606608e-02, -5.8281336e-02]],

        [[ 2.7932882e-02, -8.7643631e-02,  1.7155774e-02, ...,
           8.2015470e-02, -7.3620394e-02, -8.5265361e-02],
         [ 6.4643964e-02,  8.3204657e-02, -8.5551061e-02, ...,
          -1.7823681e-02, -8.2975566e-02, -6.9040731e-03],
         [ 5.7288036